In [15]:
import sys  
import os
from os import listdir
from os.path import isfile, join
from pathlib import Path
import json
import io
import base64
import numpy as np
import PIL
from PIL import Image
from shapely.geometry import Polygon
import rasterio.features
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from tqdm import tqdm

In [8]:
def img_b64_to_arr(img_b64):
    f = io.BytesIO()
    f.write(base64.b64decode(img_b64))
    img_arr = np.array(Image.open(f))
    return img_arr

In [26]:
# Set a path for the dataset
datasetLocation = (Path('../dataset/dfl-bl-annotated-frames/'))
datasetLocation = datasetLocation.resolve()

outputLocation = (Path('../dataset/dfl-bl-annotated-frames-seg/'))
outputLocation = outputLocation.resolve()

In [10]:
onlyFiles = [f for f in listdir(datasetLocation) if isfile(join(datasetLocation, f))]
jsonFiles = [f for f in onlyFiles if '.json' in f]
print(f'Number of json files found: {len(jsonFiles)}')

Number of json files found: 107


In [55]:
for json_file in tqdm(jsonFiles):
    jsonLoc = datasetLocation/json_file
    jsonFile = open (jsonLoc, "r")
    jsonFile = json.load(jsonFile)
    
    # Polygon for field 
    label1 = [i for i in jsonFile['shapes'] if i['label']=='1']
    field_poly = label1[0]['points']

    
    # original image
    im_b64 = jsonFile['imageData']
    im_np = img_b64_to_arr(im_b64)
    im = Image.fromarray(im_np)
    im = im.save(str(outputLocation/'img'/jsonLoc.stem) + '.png')
    # mask 
    poly = Polygon(field_poly)
    im_mask_np = rasterio.features.rasterize([poly], out_shape=(im_np.shape[0],im_np.shape[1]))
    im_mask = Image.fromarray(im_mask_np)
    im_mask = im_mask.save(str(outputLocation/'mask'/jsonLoc.stem) + '.png')

100%|███████████████████████████████████████████████████████████████████████| 107/107 [02:13<00:00,  1.25s/it]
